In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.max_columns = None
# pd.set_option('display.max_colwidth', None)
# from IPython.display import display, HTML
# display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

In [ ]:
# !ls data/

In [ ]:
# !ls sup_info/

## Functions

In [ ]:
# created append_anomaly_counts function to get anomaly counts with different datasets & slices
def append_anomaly_counts(df_info=None, dataframe=None, col_suffix=""):
    res = []
    df_info["nunique"] = dataframe.nunique().values
    df_info["uknown_count"] = dataframe.eq("?").sum().values
    for idx, row in df_info.iterrows():
        column = row["column_name"]
        if row["column_type"] == "continuous":
            dataframe[column] = dataframe[column].astype(float)
            res.append({"column_name":column, "NIU_count":0})
        else:
            dataframe[column] = dataframe[column].astype(str)
            dataframe_NIU_mask = dataframe[column].str.contains("Not in universe")
            if dataframe_NIU_mask.any():
                dataframe_NIU_uval = dataframe[column][dataframe_NIU_mask].unique().item()
                NIU_count = dataframe[dataframe[column] == dataframe_NIU_uval].shape[0]
                res.append({"column_name":column, "NIU_count":NIU_count})
            else:
                res.append({"column_name":column, "NIU_count":0})
    # matching nominal column sanity check
    if (df_info[df_info["column_type"]=="nominal"]["column_name"].values == dataframe.select_dtypes(object).columns).all():
        df_info = pd.concat([df_info, pd.DataFrame(res)["NIU_count"]], axis=1)
        df_info = df_info.assign(row_count=dataframe.shape[0])
    return df_info

In [ ]:
def change_target_value(dataframe=None):
    dataframe.loc[dataframe["target"] == "- 50000", "target"] = "0"
    dataframe.loc[dataframe["target"] == "50000+", "target"] = "1"
    return dataframe

In [ ]:
def remap_cat_column(dataframe=None, column="", remap={}):
    return dataframe.replace({column:remap})

In [ ]:
def categorical_target_eda(dataframe=None, column="", order=None, plot=True):
    ct = pd.crosstab(dataframe["target"], dataframe[column], normalize="index")
    if plot:
        display(ct)
        g = sns.catplot(
            x=column,
            kind="count",
            col="target",
            order=order,
            data=dataframe,
            sharey=False,
        )
        g.set_xticklabels(rotation=60)
    return ct.T

In [ ]:
def engineer_nominal(dataframe=None, columns=None, threshold=0.01):
    res = []
    for column in columns:
        remap = {}
        ct = categorical_target_eda(dataframe=dataframe, column=column, plot=False)
        s = ct["50000+"] - ct["- 50000"]
        for category, value in zip(s.index, s):
            if value > threshold:
                remap[category] = "gt"
            elif value < -threshold:
                remap[category] = "lt"
            else:
                remap[category] = "eq"
        res.append({column:remap})
    return res

In [ ]:
def engineer_cat_columns(train_data=None, test_data=None, remaps=None, verbose=0):
    # select float columns for concatenation
    df_train_floats = train_data.select_dtypes(include=float)
    df_test_floats = test_data.select_dtypes(include=float)
    # initalize dataframes for concatenation
    df_train_remap = pd.DataFrame()
    df_test_remap = pd.DataFrame()
    # loop through list of dictionaries and remap values in categorical columns
    for remap in remaps:
        # get column from dictionary
        column = list(remap.keys())[0]
        # remap train
        df_train_temp = remap_cat_column(dataframe=train_data, column=column, remap=remap[column])
        df_train_remap = pd.concat([df_train_remap, df_train_temp[column]], axis=1)
        # remap test
        df_test_temp = remap_cat_column(dataframe=test_data, column=column, remap=remap[column])
        df_test_remap = pd.concat([df_test_remap, df_test_temp[column]], axis=1)
    
    df_train_remap = pd.concat([df_train_floats, df_train_remap, train_data["target"]], axis=1)
    df_test_remap = pd.concat([df_test_floats, df_test_remap, test_data["target"]], axis=1)

    if verbose > 0:
        display(df_train_remap.describe(include="float"))
        display(df_train_remap.describe(include="object"))
    
    return df_train_remap, df_test_remap

## Get input data

In [ ]:
df_train0 = pd.read_csv(
    filepath_or_buffer="data/census_income_learn.csv",
    header=None).drop(24,axis=1)

In [ ]:
df_test0 = pd.read_csv(
    filepath_or_buffer="data/census_income_test.csv",
    header=None).drop(24,axis=1)

In [ ]:
data_info = [
    "|   91 distinct values for attribute #0 (age) continuous",
    "|    9 distinct values for attribute #1 (class of worker) nominal",
    "|   52 distinct values for attribute #2 (detailed industry recode) nominal",
    "|   47 distinct values for attribute #3 (detailed occupation recode) nominal",
    "|   17 distinct values for attribute #4 (education) nominal",
    "| 1240 distinct values for attribute #5 (wage per hour) continuous",
    "|    3 distinct values for attribute #6 (enroll in edu inst last wk) nominal",
    "|    7 distinct values for attribute #7 (marital stat) nominal",
    "|   24 distinct values for attribute #8 (major industry code) nominal",
    "|   15 distinct values for attribute #9 (major occupation code) nominal",
    "|    5 distinct values for attribute #10 (race) nominal",
    "|   10 distinct values for attribute #11 (hispanic origin) nominal",
    "|    2 distinct values for attribute #12 (sex) nominal",
    "|    3 distinct values for attribute #13 (member of a labor union) nominal",
    "|    6 distinct values for attribute #14 (reason for unemployment) nominal",
    "|    8 distinct values for attribute #15 (full or part time employment stat) nominal",
    "|  132 distinct values for attribute #16 (capital gains) continuous",
    "|  113 distinct values for attribute #17 (capital losses) continuous",
    "| 1478 distinct values for attribute #18 (dividends from stocks) continuous",
    "|    6 distinct values for attribute #19 (tax filer stat) nominal",
    "|    6 distinct values for attribute #20 (region of previous residence) nominal",
    "|   51 distinct values for attribute #21 (state of previous residence) nominal",
    "|   38 distinct values for attribute #22 (detailed household and family stat) nominal",
    "|    8 distinct values for attribute #23 (detailed household summary in household) nominal",
    "|   10 distinct values for attribute #24 (migration code-change in msa) nominal",
    "|    9 distinct values for attribute #25 (migration code-change in reg) nominal",
    "|   10 distinct values for attribute #26 (migration code-move within reg) nominal",
    "|    3 distinct values for attribute #27 (live in this house 1 year ago) nominal",
    "|    4 distinct values for attribute #28 (migration prev res in sunbelt) nominal",
    "|    7 distinct values for attribute #29 (num persons worked for employer) continuous",
    "|    5 distinct values for attribute #30 (family members under 18) nominal",
    "|   43 distinct values for attribute #31 (country of birth father) nominal",
    "|   43 distinct values for attribute #32 (country of birth mother) nominal",
    "|   43 distinct values for attribute #33 (country of birth self) nominal",
    "|    5 distinct values for attribute #34 (citizenship) nominal",
    "|    3 distinct values for attribute #35 (own business or self employed) nominal",
    "|    3 distinct values for attribute #36 (fill inc questionnaire for veteran's admin) nominal",
    "|    3 distinct values for attribute #37 (veterans benefits) nominal",
    "|   53 distinct values for attribute #38 (weeks worked in year) continuous",
    "|    2 distinct values for attribute #39 (year) nominal",
]

## Clean data

In [ ]:
s_data_info = pd.Series(data_info)\
    .str.replace("|", "")\
    .str.replace("distinct values for attribute #", ",")\
    .str.replace("(", ",")\
    .str.replace(")", ",")\
    .str.replace("'","")\
    .str.strip()
df_data_info = s_data_info.str.split(",", expand=True).drop(1,axis=1)
df_data_info.columns = ["nunique", "column_name", "column_type"]
df_data_info["nunique"] = df_data_info["nunique"].astype(int)
df_data_info.loc[40] = [2, "target", "nominal"]
df_data_info = df_data_info.map(lambda x: x.strip() if isinstance(x, str) else x)

#### train data clean
- df_train1

In [ ]:
print(f"inital shape: {df_train0.shape}")
print(f"number of dups: {df_train0.duplicated().sum()}") # different total than metadata file (46627 vs.46716)
if (df_train0.nunique().reset_index(drop=True) == df_data_info["nunique"]).all():
    print("renaming columns\n")
    df_train0.columns = df_data_info["column_name"].tolist()
df_train0 = df_train0.map(lambda x: x.strip() if isinstance(x, str) else x)
df_train0["target"] = df_train0["target"].str.replace(".", "")

# drop duplicate rows
df_train1 = df_train0.drop_duplicates(ignore_index=True)
print(f"shape after drop dups: {df_train1.shape}")

# if edu is Children then target < 50k
print("\nfilter Children - target counts")
print(df_train1[df_train1["education"]=="Children"]["target"].value_counts())

df_train1 = df_train1[df_train1["education"]!="Children"].reset_index(drop=True)
print(f"\nshape after drop Children: {df_train1.shape}")
# print(df_train1.duplicated().sum())

df_info_train = append_anomaly_counts(df_info=df_data_info, dataframe=df_train1)

print("\ntarget distribution")
print(df_train1["target"].value_counts())

#### Test data clean
- df_test1

In [ ]:
print(f"inital shape: {df_test0.shape}")
print(f"number of dups: {df_test0.duplicated().sum()}")
print("renaming columns\n")
df_test0.columns = df_data_info["column_name"].tolist()
df_test0 = df_test0.map(lambda x: x.strip() if isinstance(x, str) else x)
df_test0["target"] = df_test0["target"].str.replace(".", "")

# drop duplicate rows
df_test1 = df_test0.drop_duplicates(ignore_index=True)
print(f"shape after drop dups: {df_test1.shape}")

# if edu is Children then target < 50k
print("\nfilter Children - target counts")
print(df_test1[df_test1["education"]=="Children"]["target"].value_counts())

df_test1 = df_test1[df_test1["education"]!="Children"].reset_index(drop=True)
print(f"\nshape after drop Children: {df_test1.shape}")
# print(df_test1.duplicated().sum())

df_info_test = append_anomaly_counts(df_info=df_data_info, dataframe=df_test1)

print("\ntarget distribution")
print(df_test1["target"].value_counts())

## Feature engineering

In [ ]:
# filter nominal columns and exclude target
df_nominal = df_info_train[(df_info_train["column_type"] == "nominal") & (df_info_train["column_name"] != "target")]

# filtered column lists - nominal columns (nc)
nc_all = df_nominal["column_name"].values
nc_noUnknown = df_nominal[df_nominal["uknown_count"] == 0]["column_name"].values
nc_noNIU = df_nominal[df_nominal["NIU_count"] == 0]["column_name"].values
nc_noUnknownORnoNIU = df_nominal[(df_nominal["uknown_count"] == 0) & (df_nominal["NIU_count"] == 0)]["column_name"].values

res_nc_all = engineer_nominal(dataframe=df_train1, columns=nc_all, threshold=0.01)
print(f"nc_all--dataframe shape: {nc_all.shape[0]} | result column size: {len(res_nc_all)}")
res_nc_noUnknown = engineer_nominal(dataframe=df_train1, columns=nc_noUnknown, threshold=0.01)
print(f"nc_noUnknown--dataframe shape: {nc_noUnknown.shape[0]} | result column size: {len(res_nc_noUnknown)}")
res_nc_noNIU = engineer_nominal(dataframe=df_train1, columns=nc_noNIU, threshold=0.01)
print(f"nc_noNIU--dataframe shape: {nc_noNIU.shape[0]} | result column size: {len(res_nc_noNIU)}")
res_nc_noUnknownORnoNIU = engineer_nominal(dataframe=df_train1, columns=nc_noUnknownORnoNIU, threshold=0.01)
print(f"nc_noUnknownORnoNIU--dataframe shape: {nc_noUnknownORnoNIU.shape[0]} | result column size: {len(res_nc_noUnknownORnoNIU)}")

#### remap categories

In [ ]:
df_train_remap, df_test_remap = engineer_cat_columns(
    train_data=df_train1,
    test_data=df_test1,
    remaps=res_nc_all,
    verbose=0,
)

## Model data

In [ ]:
df_train2 = change_target_value(dataframe=df_train_remap)
df_test2 = change_target_value(dataframe=df_test_remap)

In [ ]:
df_train2["target"].value_counts()

In [ ]:
df_test2["target"].value_counts()

In [ ]:
# # BALANCE DATA - NOT USED YET...
# df_class0 = df_train2[df_train2["target"] == "0"]
# df_class1 = df_train2[df_train2["target"] == "1"]

# df_class0_sample = df_class0.sample(n=df_class1.shape[0], random_state=42, axis=0)
# df_train_balance = pd.concat([df_class0_sample, df_class1], axis=0).sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
X_train0 = df_train2.drop("target", axis=1)
y_train0 = df_train2["target"]

In [ ]:
X_test0 = df_test2.drop("target", axis=1)
y_test0 = df_test2["target"]

In [ ]:
matching_cols = X_train0.select_dtypes(object).nunique() == X_test0.select_dtypes(object).nunique()
cat_features = matching_cols[matching_cols==True].index.tolist()

In [ ]:
num_features = [
    "age",
    "dividends from stocks",
    "num persons worked for employer", ###
    "weeks worked in year", ###
]
features = num_features + cat_features
X_train1 = pd.get_dummies(X_train0.loc[:, features])
X_test1 = pd.get_dummies(X_test0.loc[:, features])
print(X_train1.shape)
print(X_test1.shape)

In [ ]:
from sklearn.feature_selection import mutual_info_classif as MIC

In [ ]:
mi_score = MIC(X_train1, y_train0, random_state=42)
col_MIC_mask = mi_score > np.percentile(a=mi_score, q=75)
selected_features = X_train1.columns[col_MIC_mask].tolist()

In [ ]:
X_train1 = X_train1.loc[:, selected_features]
X_test1 = X_test1.loc[:, selected_features]
print(X_train1.shape)
print(X_test1.shape)

### logistic regression - baseline model

- scale data
- explore thresholds

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay, auc, classification_report, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
model = LogisticRegression(max_iter=10000, class_weight="balanced", random_state=42)
model.fit(X_train1, y_train0)

In [ ]:
def predictions(X=None, model=None):
    y_pred = model.predict(X)
    y_probas = model.predict_proba(X)
    return y_probas, y_pred
yhat, y_pred = predictions(X=X_train1, model=model)

In [ ]:
pos_probs = yhat[:, 1]
precision, recall, threshold = precision_recall_curve(y_train0.astype(int), pos_probs)
auc(recall, precision)

In [ ]:
isplay = PrecisionRecallDisplay.from_predictions(
    y_test0, y_score, name="LinearSVC", plot_chance_level=True
)
_ = display.ax_.set_title("2-class Precision-Recall curve")

In [ ]:
display = PrecisionRecallDisplay.from_estimator(
    model, X_test1, y_test0, name="LogReg", plot_chance_level=True
)
_ = display.ax_.set_title("2-class Precision-Recall curve")

In [ ]:
pos_probs = yhat[:, 1]
precision, recall, _ = precision_recall_curve(y_train0.astype(int), pos_probs)
auc_score = auc(recall, precision)
print(f"prauc: {auc_score}")
print(classification_report(y_train0, y_pred))    

In [ ]:
y_pred = model.predict(X_train1)
yhat = model.predict_proba(X_train1)
pos_probs = yhat[:, 1]
precision, recall, _ = precision_recall_curve(y_train0.astype(int), pos_probs)
auc_score = auc(recall, precision)
# print(confusion_matrix(y_train0, y_pred))
print(f"prauc: {auc_score}")
print(classification_report(y_train0, y_pred))

In [ ]:
disp = ConfusionMatrixDisplay(
    confusion_matrix=confusion_matrix(y_train0, y_pred, labels=model.classes_),
    display_labels=model.classes_
)
disp.plot(cmap="Blues", values_format="", colorbar=False)

In [ ]:
y_pred = model.predict(X_test1)
yhat = model.predict_proba(X_test1)
pos_probs = yhat[:, 1]
precision, recall, _ = precision_recall_curve(y_test0.astype(int).values, pos_probs)
auc_score = auc(recall, precision)
print(f"prauc: {auc_score}")
# print(confusion_matrix(y_test0, y_pred))
print(classification_report(y_test0, y_pred))

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier(
    n_estimators=500,
    max_depth=10,
    min_samples_leaf=2,
    max_features=0.3,
    random_state=42,
    class_weight="balanced",
    criterion="gini"
)
model.fit(X_train1, y_train0)

In [ ]:
y_pred = model.predict(X_train1)
yhat = model.predict_proba(X_train1)
pos_probs = yhat[:, 1]
precision, recall, _ = precision_recall_curve(y_train0.astype(int).values, pos_probs)
auc_score = auc(recall, precision)
print(f"prauc: {auc_score}")
# print(confusion_matrix(y_train0, y_pred))
print(classification_report(y_train0, y_pred))

In [ ]:
y_pred = model.predict(X_test1)
yhat = model.predict_proba(X_test1)
pos_probs = yhat[:, 1]
precision, recall, _ = precision_recall_curve(y_test0.astype(int).values, pos_probs)
auc_score = auc(recall, precision)
print(f"prauc: {auc_score}")
# print(confusion_matrix(y_test0, y_pred))
print(classification_report(y_test0, y_pred))

## Single category column EDA

In [ ]:
# df_train1.describe(include="object").drop(["education", "tax filer stat", "detailed household and family stat"], axis=1)

In [ ]:
col = "detailed household summary in household"
# print(df_train1[col].value_counts())
ct = categorical_target_eda(dataframe=df_train1, column=col, plot=False)
s = ct["50000+"] - ct["- 50000"]

In [ ]:
remap = {}
threshold = 0.01
for category, value in zip(s.index, s):
    if value > threshold:
        remap[category] = "gt"
    elif value < -threshold:
        remap[category] = "lt"
    else:
        remap[category] = "eq"

df_train_remap = remap_cat_column(dataframe=df_train1, column=col, remap=remap)
ct = categorical_target_eda(dataframe=df_train_remap, column=col, order=["lt", "eq", "gt"])
df_test_remap = remap_cat_column(dataframe=df_test1, column=col, remap=remap)
ct = categorical_target_eda(dataframe=df_test_remap, column=col, order=["lt", "eq", "gt"])

## EDA

### descriptive statistics

In [ ]:
df_train1.describe()

In [ ]:
df_train1.describe(include="object")

### plot categorical distributions

In [ ]:
cat_cols = df_train1.select_dtypes(include='object')

In [ ]:
for col in cat_cols:
    n = df_train1[col].nunique()
    if n <= 22:
        sns.countplot(
            y=col,
            data=df_train1,
            hue=col,
            palette=sns.color_palette(palette="colorblind", n_colors=n),
            legend=False
        )
        plt.show()

### slice target by categorical features

In [ ]:
for col in cat_cols:
    if df_train1[col].nunique() <=4:
        display(pd.crosstab(df_train1['target'], df_train1[col], normalize='index'))

In [ ]:
for col in cat_cols:
    if df_train1[col].nunique() <= 4:
        g = sns.catplot(x = col, kind='count', col = 'target', data=df_train1, sharey=False)
        g.set_xticklabels(rotation=60)

### slice target by numerical features

In [ ]:
num_cols = df_train1.select_dtypes(float).columns.values

In [ ]:
for col in num_cols:
    df_train1[col].hist(bins=20)
    print(col)
    plt.show()

In [ ]:
for col in num_cols:
    sns.boxplot(
        y=df_train1['target'].astype('category'),
        hue=df_train1['target'].astype('category'),
        x=col,
        data=df_train1,
        palette=sns.color_palette(palette="colorblind", n_colors=2)
    )
    plt.show()

### Group numerical features (mean) by categorical features

In [ ]:
for col in cat_cols:
    if df_train1[col].nunique() <= 3:
        display(df_train1.groupby(col)[num_cols].mean())

### Correlation matrix for numerical features

In [ ]:
corr = df_train1.select_dtypes(float).corr()
corr

In [ ]:
plt.figure(figsize=(6,6))
sns.heatmap(corr, cmap='RdBu_r', annot=True, vmax=1, vmin=-1)
plt.show()